In [ ]:
% matplotlib inline
import numpy as np
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# Misión de medición de lluvia tropical (Tropical Rainfall Measuring Mission)

## Importación de datos

In [ ]:
!ls data | head -15

In [ ]:
ds = xr.open_dataset("./data/3B43.20081201.7A.HDF.nc")

## Inspección de los datos

In [ ]:
ds

### Ejemplo de una cuadrícula de datos

<img src="_img/gridded_data.png" width=40%/>

### Ejemplo de una cuadrícula multidimensional

<img src="_img/gridded_data_dimensions.png" width=50%/>


In [ ]:
ds

In [ ]:
ds["precipitation"]

In [ ]:
ds.precipitation

In [ ]:
print(ds.attrs["Grid.GridHeader"])

## Trazado de datos

In [ ]:
ds["precipitation"].plot();

## Trazado mejorado usando cartopy

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.Orthographic(-70,-15))
ds.precipitation.plot.contourf(ax=ax, transform=ccrs.PlateCarree());
ax.coastlines(color="white")
ax.add_feature(cartopy.feature.BORDERS, edgecolor="white")
ax.gridlines()
ax.set_extent([-82, -68.5, 0.5, -18]); #x0, x1, y0, y1

## Problema con las unidades

* los datos de precipitación se dan en mm/h $\to$ mm/mes ???

$\to$ extrae el número de días del año y del mes

### Computar mm por mes

**Encontrar el número de días por mes de interés**

In [ ]:
print(ds.attrs["FileHeader"])

In [ ]:
file_header = ds.attrs["FileHeader"]
text = file_header.split(";")[5]
text

In [ ]:
import re
match = re.search(r'\d{4}-\d{2}-\d{2}', text)
match

In [ ]:
from datetime import datetime
date = datetime.strptime(match.group(), '%Y-%m-%d').date()
date

In [ ]:
date.day

In [ ]:
date.month

In [ ]:
date.year

### Refactorización

In [ ]:
# %load src/extract_timestep.py
def extract_timestep(ds):
    import re
    import datetime
    import numpy as np
    file_header = ds.attrs["FileHeader"]
    text = file_header.split(";")[5]
    match = re.search(r'\d{4}-\d{2}-\d{2}', text)
    date = datetime.datetime.strptime(match.group(), '%Y-%m-%d').date()
    return date

In [ ]:
ds

In [ ]:
extract_timestep(ds)

In [ ]:
extract_timestep(ds).day

### Solución para el problema con unidades

In [ ]:
ds.precipitation

In [ ]:
def compute_mm_per_month_from_mm_per_hour(ds, days):
    return ds * 24 * days

In [ ]:
prec_mm_per_month = compute_mm_per_month_from_mm_per_hour(ds.precipitation, extract_timestep(ds).day)
prec_mm_per_month

In [ ]:
prec_mm_per_month.plot()

### Trazado mejorado usando cartopy

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.Orthographic(-70,-15))
prec_mm_per_month.plot.contourf(ax=ax, transform=ccrs.PlateCarree());
ax.coastlines(color="white")
ax.add_feature(cartopy.feature.BORDERS, edgecolor="white")
ax.gridlines()
ax.set_extent([-82, -68.5, 0.5, -18]); #x0, x1, y0, y1

***

## Deberes

>* Vaya al repositorio en línea ([enlace](https://github.com/eotp/diplomado-internacional-GIRH)) e intente reproducir el análisis.